# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [3]:
from utils import setup_db_context
import pandas as pd
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking

데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [4]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_ret)}")
df_ret.head()

가져온 거래 수: 5
총 레코드 수: 235671


,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
0,2,2025,11320,도봉구,10600,방학동,1,대지,0638,0010,...,14800,37.66,16.0,4,,,1994.0,연립다세대,중개거래,서울 도봉구
1,3,2025,11380,은평구,10800,역촌동,,,,,...,39620,74.08,182.0,,,,1972.0,단독다가구,직거래,
2,4,2025,11500,강서구,10200,등촌동,1,대지,0628,0013,...,10300,26.16,35.71,6,,,1999.0,오피스텔,중개거래,서울 강서구
3,5,2025,11620,관악구,10200,신림동,1,대지,1639,0051,...,12700,18.70,26.16,8,,,2014.0,오피스텔,중개거래,서울 관악구
4,6,2025,11530,구로구,10600,고척동,1,대지,0339,0000,...,84000,59.89,0.0,9,,,2009.0,아파트,중개거래,"서울 구로구, 서울 양천구"


## 2. 공영주차장 (Public Parkings)

In [5]:
# SQLAlchemy를 사용하여 데이터 가져오기
parkings = PublicParking.query.limit(5).all()
print(f"가져온 주차장 레코드 수: {len(parkings)}")

# DataFrame으로 변환
stmt = select(PublicParking)
df_pp = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_pp)}")
df_pp.head()

가져온 주차장 레코드 수: 5
총 레코드 수: 1875


,pp_id,parking_code,parking_name,address,parking_type,parking_type_name,operation_type,operation_type_name,phone_number,parking_status_available,...,bus_basic_time_min,bus_add_rate,bus_add_time_min,day_max_rate,lat,lng,share_parking_company_name,share_parking,share_parking_company_link,share_parking_etc
0,1,171721,세종로 공영주차장(시),종로구 세종로 80-1,NW,노외 주차장,1,시간제 주차장,02-2290-6566,1,...,0,0,0,30900,37.573403,126.975884,,N,,
1,2,171730,종묘주차장 공영주차장(시),종로구 훈정동 2-0,NW,노외 주차장,1,시간제 주차장,02-2290-6166,1,...,0,0,0,28800,37.571504,126.994969,,N,,
2,3,171900,훈련원공원 공영주차장(시),중구 을지로5가 40-3,NW,노외 주차장,1,시간제 주차장,02-3405-4597,1,...,0,0,0,39600,37.567400,127.003521,,N,,
3,4,172051,한강진역 공영주차장(시),용산구 한남동 728-27,NW,노외 주차장,1,시간제 주차장,02-795-6406,1,...,0,0,0,28800,37.539522,127.002580,,N,,
4,5,172065,용산주차빌딩 공영주차장(시),용산구 한강로2가 12-9,NW,노외 주차장,1,시간제 주차장,02-2290-6014,1,...,0,0,0,21600,37.534364,126.965418,,N,,


In [6]:
selected_cols = ['parking_name', 'address', 'parking_type_name','total_spaces','current_parking','basic_rate'] #필요한 컬럼을 가지고 데이터 프레임 생성
parking_df = df_pp[selected_cols]

parking_nw = parking_df[parking_df['parking_type_name'].str.contains('노외 주차장')] 
parking_nw.sort_values(by='total_spaces', ascending=False).reset_index(drop=True)        # 주차장(노외) 통합 데이터 총 주차면수 순서대로       

parking_nw

,parking_name,address,parking_type_name,total_spaces,current_parking,basic_rate
0,세종로 공영주차장(시),종로구 세종로 80-1,노외 주차장,1260,655,430
1,종묘주차장 공영주차장(시),종로구 훈정동 2-0,노외 주차장,1317,960,400
2,훈련원공원 공영주차장(시),중구 을지로5가 40-3,노외 주차장,873,641,550
3,한강진역 공영주차장(시),용산구 한남동 728-27,노외 주차장,174,105,400
4,용산주차빌딩 공영주차장(시),용산구 한강로2가 12-9,노외 주차장,561,305,300
...,...,...,...,...,...,...
1870,남산파출소공영주차장(시),중구 회현동1가 1-16,노외 주차장,13,0,330
1871,용산구 한강로2가 2-39(시),용산구 한강로2가 2-39,노외 주차장,12,0,4000
1872,삼성유료주차장(시),강남구 신사동 625-13,노외 주차장,20,0,3000
1873,서울암사동유적 주차장(시),강동구 암사동 139-2,노외 주차장,108,0,1000


In [7]:
parking_nss = parking_df[parking_df['parking_type_name'].str.contains('노상 주차장')] # 노상주차장은 코드 카운팅 해서 총 주차면수 정해야함

parking_nss['address_count'] = parking_nss.groupby('address')['address'].transform('count') # 중복되는 address를 count하여 새로운 컬럼을 생성 

parking_nss['total_spaces'] = parking_nss['address_count'] #위 새롭게 생성된 컬럼을 total_spaces로 바꿔줌
parking_nss.drop(columns='address_count',inplace=True)
parking_ns = parking_nss.copy()

parking_ns.drop_duplicates(subset=['address'], inplace=True)

parking_ns = parking_ns.sort_values(by='total_spaces', ascending=False).reset_index(drop=True)  # 주차장(노상) 통합 데이터 총 주차면수 순서대로

parking_ns

C:\Users\User\AppData\Local\Temp\ipykernel_12940\1668448781.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parking_nss['address_count'] = parking_nss.groupby('address')['address'].transform('count') # 중복되는 address를 count하여 새로운 컬럼을 생성
C:\Users\User\AppData\Local\Temp\ipykernel_12940\1668448781.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parking_nss['total_spaces'] = parking_nss['address_count'] #위 새롭게 생성된 컬럼을 total_spaces로 바꿔줌
C:\Users\User\AppData\Local\Temp\ipykernel_12940\1668448781.py:6: Set

,parking_name,address,parking_type_name,total_spaces,current_parking,basic_rate
0,여의서로 공영주차장(시),영등포구 여의도동 8-1,노상 주차장,84,38,160
1,을지로 공영주차장(시),중구 을지로3가 282-8,노상 주차장,80,54,750
2,청계8가(남) 공영주차장(시),성동구 상왕십리동 12-47,노상 주차장,72,40,450
3,봉천복개3 공영주차장(시),관악구 신림동 1467-3,노상 주차장,61,34,220
4,남대문시장2번게이트앞 이륜차 주차장(시),중구 남대문로4가 3-6 0,노상 주차장,61,0,0
...,...,...,...,...,...,...
79,세종대로3 관광버스 승하차 허용 구간(시),중구 태평로2가 113-1 (신협 ~ 세븐일레븐),노상 주차장,2,0,0
80,APM플레이스 앞 관광버스 주차허용 구간(시),중구 을지로7가 112-3,노상 주차장,2,0,0
81,세종대로2 관광버스 승하차 허용 구간(시),중구 정동 5-5 (정동 5-5(시청역 2번 출구 앞),노상 주차장,2,0,0
82,이태원 입구 관광버스 승하차 허용 구간(시),용산구 이태원동 56-4,노상 주차장,2,0,


In [8]:
parking = pd.concat([parking_nw, parking_ns], ignore_index=True)
parking = parking.sort_values(by='total_spaces', ascending=False).reset_index(drop=True)    # 주차장(노외, 노상) 통합 데이터 총 주차면수 순서대로

parking['Area_Gu'] = parking['address'].str.split().str[0]
parking['Area_Dong'] = parking['address'].str.split().str[1]        # address 주소를 split해서 Gu, Dong별로 컬럼을 생성해줌

parking['available_spaces'] = (parking['total_spaces'] - parking['current_parking'])    # 총 주차면수 - 주차차량수 = 주차가능한 공간을 컬럼으로 생성 
parking['available_spaces'] = parking['available_spaces'].astype(int)                   # 실수형 컬럼을 정수형으로 변환함

parking

,parking_name,address,parking_type_name,total_spaces,current_parking,basic_rate,Area_Gu,Area_Dong,available_spaces
0,천호역 공영주차장(시),강동구 천호동 455-0,노외 주차장,1431,1376,260,강동구,천호동,55
1,종묘주차장 공영주차장(시),종로구 훈정동 2-0,노외 주차장,1317,960,400,종로구,훈정동,357
2,세종로 공영주차장(시),종로구 세종로 80-1,노외 주차장,1260,655,430,종로구,세종로,605
3,파미에(반포천) 주차장(시),서초구 반포동 118-3,노외 주차장,1186,15,440,서초구,반포동,1171
4,동대문 공영주차장(시),중구 신당동 251-7,노외 주차장,1092,270,360,중구,신당동,822
...,...,...,...,...,...,...,...,...,...
150,세종대로3 관광버스 승하차 허용 구간(시),중구 태평로2가 113-1 (신협 ~ 세븐일레븐),노상 주차장,2,0,0,중구,태평로2가,2
151,APM플레이스 앞 관광버스 주차허용 구간(시),중구 을지로7가 112-3,노상 주차장,2,0,0,중구,을지로7가,2
152,세종대로2 관광버스 승하차 허용 구간(시),중구 정동 5-5 (정동 5-5(시청역 2번 출구 앞),노상 주차장,2,0,0,중구,정동,2
153,이태원 입구 관광버스 승하차 허용 구간(시),용산구 이태원동 56-4,노상 주차장,2,0,,용산구,이태원동,2


In [16]:
input_name = "도봉구"

def filter_by_district(df, input_name):
    return df[df['Area_Gu']==input_name].copy()

result=filter_by_district(parking, input_name)
result

,parking_name,address,parking_type_name,total_spaces,current_parking,basic_rate,Area_Gu,Area_Dong,available_spaces
14,도봉산역 공영주차장(시),도봉구 도봉동 288-19,노외 주차장,358,383,170,도봉구,도봉동,-25
25,도봉산 공영주차장(시),도봉구 도봉동 282-26,노외 주차장,142,89,170,도봉구,도봉동,53
30,창동역(서) 공영주차장(시),도봉구 창동 330-0,노외 주차장,106,101,260,도봉구,창동,5


In [31]:
input_district_name = '도봉구'
input_dong_name = '도봉동'

def filter_by_district(df, input_district_name):
        return df[df['Area_Gu']==input_district_name].copy()

result = filter_by_district(parking, input_district_name)

if input_dong_name is not None:
    result = result[result['Area_Dong'] == input_dong_name]
    
#result=result.to_dict()

result

,parking_name,address,parking_type_name,total_spaces,current_parking,basic_rate,Area_Gu,Area_Dong,available_spaces
14,도봉산역 공영주차장(시),도봉구 도봉동 288-19,노외 주차장,358,383,170,도봉구,도봉동,-25
25,도봉산 공영주차장(시),도봉구 도봉동 282-26,노외 주차장,142,89,170,도봉구,도봉동,53
